# データ・サイエンス	データ・サイエンス社会応用論 / ICT社会応用演習Ⅳ
## 14-2. 地震情報を抽出しよう

### 1. 気象庁防災情報XMLの読み込み
#### 1-a) 下準備

In [8]:
import xml.etree.ElementTree as ET
from dateutil import parser
import requests
import pandas as pd
import time

In [9]:
EQVOL_LONG_FEED = 'https://www.data.jma.go.jp/developer/xml/feed/eqvol_l.xml'

#### 1-b) フィードのダウンロード

In [10]:
response = requests.get(EQVOL_LONG_FEED)
response.encoding = 'utf-8'
print(response.text)

<?xml version="1.0" encoding="utf-8"?>
<feed xmlns="http://www.w3.org/2005/Atom" lang="ja">
  <title>長期（地震火山）</title>
  <subtitle>JMAXML publishing feed</subtitle>
  <updated>2025-01-22T17:17:53+09:00</updated>
  <id>https://www.data.jma.go.jp/developer/xml/feed/eqvol.xml#long_1737533873</id>
  <link rel="related" href="https://www.jma.go.jp/"/>
  <link rel="self" href="https://www.data.jma.go.jp/developer/xml/feed/eqvol.xml"/>
  <link rel="hub" href="http://alert-hub.appspot.com/"/>
  <rights type="html"><![CDATA[
<a href="https://www.jma.go.jp/jma/kishou/info/coment.html">利用規約</a>,
<a href="https://www.jma.go.jp/jma/en/copyright.html">Terms of Use</a>
]]></rights>
  <entry>
    <title>噴火に関する火山観測報</title>
    <id>https://www.data.jma.go.jp/developer/xml/data/20250122081703_0_VFVO52_400000.xml</id>
    <updated>2025-01-22T08:17:03Z</updated>
    <author>
      <name>福岡管区気象台　鹿児島地方気象台</name>
    </author>
    <link type="application/xml" href="https://www.data.jma.go.jp/developer/xml/dat

#### 1-c) DataFrameの作成

In [11]:
NS = {'': 'http://www.w3.org/2005/Atom'}

root = ET.fromstring(response.content)

rows = []
for entry in root.findall('entry', NS):
    rows.append({
        'title' : entry.find('title', NS).text,
        'content' : entry.find('content', NS).text,
        'updated' : parser.parse(entry.find('updated', NS).text),
        'link' : entry.find('link', NS).get('href')
    })

df_feed = pd.DataFrame(rows)
df_feed

,title,content,updated,link
0,噴火に関する火山観測報,【火山名 桜島 噴火に関する火山観測報】火 山：桜島\n日 時：2025年01月22日1...,2025-01-22 08:17:03+00:00,https://www.data.jma.go.jp/developer/xml/data/...
1,推定噴煙流向報,【桜島 推定噴煙流向報】,2025-01-22 08:17:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
2,噴火に関する火山観測報,【火山名 桜島 噴火に関する火山観測報】火 山：桜島\n日 時：2025年01月22日1...,2025-01-22 08:13:09+00:00,https://www.data.jma.go.jp/developer/xml/data/...
3,推定噴煙流向報,【桜島 推定噴煙流向報】,2025-01-22 08:13:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
4,降灰予報（定時）,【火山名 諏訪之瀬島 降灰予報（定時）】 現在、諏訪之瀬島は噴火警戒レベル２（火口周辺規制）...,2025-01-22 08:00:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
...,...,...,...,...
557,降灰予報（定時）,【火山名 霧島山（新燃岳） 降灰予報（定時）】 現在、霧島山（新燃岳）は噴火警戒レベル２（火...,2025-01-15 11:00:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
558,降灰予報（定時）,【火山名 岩手山 降灰予報（定時）】 現在、岩手山は噴火警戒レベル２（火口周辺規制）です。岩...,2025-01-15 11:00:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
559,降灰予報（定時）,【火山名 浅間山 降灰予報（定時）】 現在、浅間山は噴火警戒レベル２（火口周辺規制）です。浅...,2025-01-15 11:00:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...
560,降灰予報（定時）,【火山名 薩摩硫黄島 降灰予報（定時）】 現在、薩摩硫黄島は噴火警戒レベル２（火口周辺規制）...,2025-01-15 11:00:00+00:00,https://www.data.jma.go.jp/developer/xml/data/...


In [12]:
df_feed.groupby('title')[['title']].count()

,title
title,
噴火に関する火山観測報,22
噴火警報・予報,1
推定噴煙流向報,22
火山の状況に関する解説情報,18
降灰予報（定時）,439
震度速報,5
震源に関する情報,5
震源・震度に関する情報,50


#### 1-d) 地震情報での絞り込み

In [13]:
df_eq_feed = df_feed[df_feed['title'] == '震源・震度に関する情報']
df_eq_feed

,title,content,updated,link
23,震源・震度に関する情報,【震源・震度情報】２２日１３時３７分ころ、地震がありました。,2025-01-22 04:40:19+00:00,https://www.data.jma.go.jp/developer/xml/data/...
34,震源・震度に関する情報,【震源・震度情報】２２日０９時４５分ころ、地震がありました。,2025-01-22 00:48:25+00:00,https://www.data.jma.go.jp/developer/xml/data/...
43,震源・震度に関する情報,【震源・震度情報】２２日０６時０６分ころ、地震がありました。,2025-01-21 21:09:34+00:00,https://www.data.jma.go.jp/developer/xml/data/...
54,震源・震度に関する情報,【震源・震度情報】２２日０３時２９分ころ、地震がありました。,2025-01-21 18:32:49+00:00,https://www.data.jma.go.jp/developer/xml/data/...
55,震源・震度に関する情報,【震源・震度情報】２２日０３時２６分ころ、地震がありました。,2025-01-21 18:28:38+00:00,https://www.data.jma.go.jp/developer/xml/data/...
56,震源・震度に関する情報,【震源・震度情報】２２日０３時２２分ころ、地震がありました。,2025-01-21 18:25:27+00:00,https://www.data.jma.go.jp/developer/xml/data/...
57,震源・震度に関する情報,【震源・震度情報】２２日０３時０３分ころ、地震がありました。,2025-01-21 18:06:46+00:00,https://www.data.jma.go.jp/developer/xml/data/...
66,震源・震度に関する情報,【震源・震度情報】２２日００時１１分ころ、地震がありました。,2025-01-21 15:14:40+00:00,https://www.data.jma.go.jp/developer/xml/data/...
75,震源・震度に関する情報,【震源・震度情報】２１日２２時３１分ころ、地震がありました。,2025-01-21 13:34:23+00:00,https://www.data.jma.go.jp/developer/xml/data/...
76,震源・震度に関する情報,【震源・震度情報】２１日２２時０１分ころ、地震がありました。,2025-01-21 13:04:21+00:00,https://www.data.jma.go.jp/developer/xml/data/...


### 2. 地震情報の取得
#### 2-a) 地震情報のエントリを解析する関数

In [14]:
def parse_earthquake(eq_feed):
    response = requests.get(eq_feed)
    response.encoding = 'utf-8'
    NS = {
        '': 'http://xml.kishou.go.jp/jmaxml1/body/seismology1/',
        'jmx_eb': 'http://xml.kishou.go.jp/jmaxml1/elementBasis1/'
    }
    
    root = ET.fromstring(response.content)
    earthquake = root.find('Body', NS).find('Earthquake', NS)
    time = parser.parse(earthquake.find('OriginTime', NS).text)
    area = earthquake.find('Hypocenter', NS).find('Area', NS)
    area_name = area.find('Name', NS).text
    area_coord = area.find('jmx_eb:Coordinate', NS).text
    magnitude = earthquake.find('jmx_eb:Magnitude', NS).text

    return {
        'time' : time,
        'area_name': area_name,
        'area_coord': area_coord,
        'magnitude': magnitude
    }

#### 2-b) 地震情報から1件ずつデータを取得

In [15]:
rows = []
for eq_feed in df_eq_feed['link']:
    rows.append(parse_earthquake(eq_feed))
    time.sleep(0.5)
df_eq = pd.DataFrame(rows)
df_eq

,time,area_name,area_coord,magnitude
0,2025-01-22 13:37:00+09:00,日高地方東部,+42.3+143.0-50000/,3.5
1,2025-01-22 09:45:00+09:00,山梨県中・西部,+35.6+138.7-20000/,3.4
2,2025-01-22 06:06:00+09:00,福島県会津,+37.0+139.4+0/,3.5
3,2025-01-22 03:29:00+09:00,福島県会津,+37.0+139.4-10000/,2.2
4,2025-01-22 03:25:00+09:00,福島県会津,+37.0+139.4-10000/,2.3
5,2025-01-22 03:22:00+09:00,福島県会津,+37.0+139.4-10000/,2.4
6,2025-01-22 03:03:00+09:00,福島県会津,+37.0+139.4-10000/,2.9
7,2025-01-22 00:11:00+09:00,福島県沖,+37.7+141.5-60000/,4.0
8,2025-01-21 22:31:00+09:00,日向灘,+31.9+132.0-30000/,4.1
9,2025-01-21 22:01:00+09:00,日高地方西部,+42.6+142.2-10000/,2.7
